In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("../dataset/240128_TripAdvisorHotelReviews/dataset_removed_stopwords_ver.csv")

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# 예시 데이터 (refined_df['tokens']에 토큰이 이미 들어가 있다고 가정)
# 토큰화된 데이터가 저장된 컬럼 이름이 'tokens'일 경우:
tokenized_text = df['tokens']

# 토크나이저 객체 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)

# 토큰들을 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(tokenized_text)

# 시퀀스 패딩
max_sequence_length = 200  # 시퀀스의 최대 길이 (임의 설정)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

2024-02-06 14:57:03.547876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
import torchtext.vocab

glove = torchtext.vocab.GloVe(name='840B', dim=512) # 이미 학습된 것을 불러오기만 하는 것임. torchtext의 vocab에 존재.
# hidden size를 몇으로 정할래이다. V -> hidden (100 / 300 / 512 / 256)
print(f'There are {len(glove.itos)} words in the vocabulary')
# itos -> dictionary의 사이즈가 무엇이냐는 것.
# 우리는 지금 아래와 같이 400000개의 words를 사용 가능함.

.vector_cache/glove.840B.300d.zip: 8.19kB [00:00, 46.4kB/s]


BadZipFile: File is not a zip file